In [1]:
import pandas as pd

In [47]:
# step 1 in cleaning the pdf: replace $ and ) with nothing and ( with a negative sign
pdf_data_2015 = pd.read_csv("input/2015-Benefit-Year-RI-RA.csv")
excel_data_2015 = pd.read_csv("input/2015_Part1_2_Summary_Data_Premium_Claims.csv")

In [48]:
from re import sub
from decimal import Decimal

def strip_money_values( column ):
    i = 0
    for money in pdf_data_2015[column]:
        if money == '$-' or money == 'N/E' or money == 'N/A_MA_Issuer' or money == 'N/A_DefaultCharge':
            pdf_data_2015[column][i] = "N/A"
        else:
            value = Decimal(sub(r'[^\d.]', '', money))
            if '(' in money:
                value = -1 * value
            else:
                value = value
            pdf_data_2015[column][i] = value
        i = i + 1
        print(money)
    print(pdf_data_2015)

columns = ["HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
                                "REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)",
                               "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)"]

for column in columns:
    strip_money_values(column)
print(pdf_data_2015)

C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\guanz\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


$(34,550.55)
$12,358,455.76
$(2,908,793.99)
$-
$(9,415,111.19)
$-
$(223,260.20)
$(3,473,430.83)
$-
$13,434,646.95
$-
$(805,865.40)
$(8,989,541.02)
$-
$-
$57,450.42
$-
$-
$(668,914.23)
$(2,751,129.36)
$-
$(280,355.58)
$(15,072.70)
$(5,029.01)
$(10,893,205.76)
$-
$6,656,762.38
$8,441,174.73
$-
$-
$(484,230.53)
$-
$856,409.75
$-
$26,294,064.75
$20,131,634.31
$-
$(48,452,998.55)
$(247,328.86)
$689,053.89
$(5,999,819.70)
$-
$-
$168,620.34
$-
$11,878,587.57
$-
$(3,663,100.18)
$2,978,136.23
$(9,792,044.00)
$784,206.91
$1,953,496.40
$(558,586.58)
$-
$2,979,667.72
$(25,983,024.62)
$-
$-
$(26,380,560.05)
$(165,775.07)
$5,953,639.83
$(82,078,859.73)
$-
$(28,712,244.96)
$-
$-
$63,308,434.85
$309,874.09
$-
$(125,635,855.94)
$-
$181,516,598.98
$(40,592.40)
$(115,235.31)
$9,791,489.44
$(23,491,264.75)
$(1,713,529.97)
$-
$53,436,905.52
$173,278.24
$(41,180,913.97)
$(7,630,792.84)
$98,569.13
$-
$6,541,514.95
$10,417,522.17
$-
$1,562,462.46
$2,379,976.29
$-
$6,064,407.53
$29,925.42
$664,060.25
$-
$(318,

$1,279,803.45
N/E
N/E
$4,303,697.77
$14,187,510.70
$0.00
$3,485,778.72
$108,444.44
$2,288,287.67
$1,369,709.18
$5,237,342.47
N/E
$0.00
N/E
N/E
$9,028,543.74
$188,485.20
$1,227,648.12
N/E
$1,534,183.55
$2,909,573.77
$750,094.76
N/E
$613,608.61
N/E
$2,392,980.67
$0.00
N/E
$46,831,049.28
$3,319,642.29
$4,415,250.76
$4,673,126.71
$125,759,129.37
N/E
$3,609.11
$3,272,853.74
N/E
$6,052,960.34
$113,550.68
N/E
$9,288,091.28
$1,959,663.44
$4,222,933.09
N/E
N/E
$3,138,358.14
$3,675,197.04
$10,203,410.75
$0.00
$4,910,519.65
$19,470,609.22
N/E
$22,865,750.61
$31,011,926.25
$28,854,403.54
N/E
N/E
N/E
$2,682,965.04
$37,693.57
N/E
N/E
$1,954,932.69
$0.00
$8,234,968.46
N/E
N/E
$1,557,641.67
$18,785,123.87
N/E
$178,090.31
$62,611.94
$4,586,657.17
$0.00
$8,893,140.98
N/E
$7,013,675.92
$2,894,124.53
$29,263,271.43
$14,667,714.03
N/E
N/E
N/E
N/E
$222,234,005.55
N/E
$293,030.60
$293,355.40
$1,230,433.68
$0.00
$0.00
$26,513,178.72
$38,747,450.73
N/E
$0.00
$651,988.51
N/E
N/E
N/E
N/E
$2,414,167.19
N/E
$12,58

$-
$900,266.16
$-
$(47,954.99)
$543,121.99
$-
$296,497.65
$355,617.92
$4,970,157.33
$(4,599,781.33)
$(35.55)
$-
$-
$(4,432,593.99)
$(1,913,246.31)
$(56,070.83)
$23,085.14
$-
$(55,802,137.54)
$765,550.26
$98,210,694.17
$-
$-
$(86,677,086.30)
$26,507,101.90
$(1,540,479.60)
$(1,290,603.41)
$358,925.34
$-
$(3,431,344.78)
$(161,102.44)
$(9,525,637.47)
$23,570.86
$19,822,043.90
$-
$-
$(5,238,158.44)
$-
$3,553,914.02
$3,006,738.38
$11,418,011.12
$-
$(819,602.44)
$951,160.48
$-
$(124,133.71)
$-
$-
$(1,520,187.65)
$(1,750,013.59)
$-
$979,323.74
$3,278,864.95
$(1,803,127.62)
$(95,294.25)
$4,347,588.59
$(1,239,886.17)
$-
$-
$(2,204,692.42)
$5,162,193.87
$(610,884.79)
$-
$(291,961.23)
$(1,834,583.56)
$(6,351.77)
$-
$-
$-
$25,341,838.53
$-
$(9,781,592.11)
$(6,459,820.49)
$(10,932,640.17)
$(586,198.35)
$(46,276.28)
$(389,192.26)
$(905,578.03)
$(171,150.38)
$(335,906.75)
$15,545,279.98
$(10,757,047.29)
$(2,940,128.99)
$(241,246.17)
$67,458.83
$(557,909.15)
$2,329,735.02
$(5,432.47)
$(1,468,781.32)
$-

In [49]:
print(pdf_data_2015)

     Unnamed: 0  HIOS ID              HIOS INPUTTED INSURANCE COMPANY NAME  \
0             0    11082                      Aetna Life Insurance Company   
1             1    38344          Premera Blue Cross Blue Shield of Alaska   
2             2    38596                            Time Insurance Company   
3             3    62637                 John Alden Life Insurance Company   
4             4    73836                            Moda Health Plan, Inc.   
5             5    80049               United Healthcare Insurance Company   
6             6    13939                            Time Insurance Company   
7             7    44580                          Humana Insurance Company   
8             8    44902                Federated Mutual Insurance Company   
9             9    46944             Blue Cross and Blue Shield of Alabama   
10           10    49771                 John Alden Life Insurance Company   
11           11    59809          United Healthcare Life Insuran

In [4]:
print(excel_data_2015)

        MR_SUBMISSION_TEMPLATE_ID                   ROW_LOOKUP_CODE  \
0                          133675                      ACA_9010_FEE   
1                          133675     ACC_MED_INC_POOLS_BON_CURR_YR   
2                          133675    ACC_MED_INC_POOLS_BON_PRIOR_YR   
3                          133675   ACTIVITES_TO_PREVENT_HOSP_READM   
4                          133675      AGNTS_AND_BROKERS_FEES_COMMS   
5                          133675     ALL_OTHER_CLAIMS_ADJ_EXPENSES   
6                          133675  ALLOWABLE_FRAUD_REDUCTION_EX_PT1   
7                          133675     ALLOWABLE_FRAUD_REDUCTION_EXP   
8                          133675          ALLOWABLE_ICD10_EXPENSES   
9                          133675                              APTC   
10                         133675           BLENDED_RATE_ADJUSTMENT   
11                         133675    CLM_ALL_INCU_DATES_PAID_REP_YR   
12                         133675        CLM_INC_REP_YR_AND_FOLL_YR   
13    

In [50]:
pdf_data_2015.to_csv(path_or_buf="input/2015-Benefit-Year-RI-RA-numeric.csv")